# TD2 Elasticsearch - Guide Explicatif

Ce TD se concentre sur la **gestion des répliques** et les **états du cluster** (green, yellow, red).

## Prérequis

Assurez-vous d'avoir installé la librairie :
```bash
pip install elasticsearch
```

Et que votre cluster Elasticsearch est démarré sur `http://localhost:9200`


In [ ]:
# Import de la librairie et connexion au cluster
from elasticsearch import Elasticsearch
from pprint import pprint

# Connexion au cluster Elasticsearch
es = Elasticsearch("http://localhost:9200")

print("✅ Connexion établie avec Elasticsearch")


## Introduction : États du cluster Elasticsearch

Elasticsearch a trois états principaux pour un cluster :

- **🟢 Green** : Tous les shards primaires et répliques sont actifs
- **🟡 Yellow** : Tous les shards primaires sont actifs, mais certaines répliques ne le sont pas
- **🔴 Red** : Certains shards primaires ne sont pas actifs

Dans ce TD, nous allons manipuler le facteur de réplication pour observer ces différents états.


## 3a. Création d'un index avec facteur de réplication 2

Créons un index "multimedia" avec 1 shard primaire et **2 répliques**. 

⚠️ **Important** : Si vous n'avez qu'un seul nœud dans votre cluster, Elasticsearch ne pourra pas créer les répliques (car une réplique doit être sur un nœud différent du shard primaire). Le cluster passera donc en état **yellow**.


In [ ]:
# Supprimer l'index s'il existe déjà (pour repartir à zéro)
if es.indices.exists(index="multimedia"):
    es.indices.delete(index="multimedia")
    print("🗑️  Index 'multimedia' supprimé")

# Créer un index avec 1 shard primaire et 2 répliques
es.indices.create(
    index="multimedia",
    body={
        "settings": {
            "index": {
                "number_of_shards": 1,
                "number_of_replicas": 2
            }
        }
    }
)
print("✅ Index 'multimedia' créé avec 1 shard et 2 répliques")


## 4a. Vérification de l'état du cluster (devrait être Yellow)

Vérifions l'état du cluster. Si vous avez un seul nœud, le cluster devrait être en état **yellow** car les 2 répliques ne peuvent pas être créées (elles nécessitent 2 nœuds supplémentaires).


In [ ]:
# Vérification de la santé du cluster
health = es.cluster.health()
print(f"État du cluster: {health['status'].upper()}")
print(f"Nombre de nœuds: {health['number_of_nodes']}")
print(f"Nombre de nœuds de données: {health['number_of_data_nodes']}")
print(f"Shards primaires actifs: {health['active_primary_shards']}")
print(f"Shards actifs: {health['active_shards']}")
print(f"Shards non assignés: {health['unassigned_shards']}")

if health['status'] == 'yellow':
    print("\n🟡 Le cluster est en état YELLOW")
    print("   → Tous les shards primaires sont actifs, mais certaines répliques ne peuvent pas être créées")
    print("   → Cela arrive souvent avec un cluster à un seul nœud")
elif health['status'] == 'green':
    print("\n🟢 Le cluster est en état GREEN")
    print("   → Tous les shards primaires et répliques sont actifs")
elif health['status'] == 'red':
    print("\n🔴 Le cluster est en état RED")
    print("   → Certains shards primaires ne sont pas actifs")

pprint(health)


## 2b. Vérification de l'état du cluster (encore Yellow)

Vérifions à nouveau l'état du cluster pour confirmer qu'il est toujours en yellow.


In [ ]:
# Vérification de la santé du cluster (encore)
health = es.cluster.health()
print(f"État du cluster: {health['status'].upper()}")
print(f"Shards non assignés: {health['unassigned_shards']}")

if health['status'] == 'yellow':
    print("\n🟡 Le cluster est toujours en état YELLOW")
    print("   → Les répliques ne peuvent toujours pas être créées")


## 3b. Mise à jour du facteur de réplication à 1

Réduisons le facteur de réplication à 1. Cela permettra au cluster de créer une réplique (si vous avez au moins 2 nœuds) ou de passer en état **green** si vous n'avez qu'un seul nœud (car 1 réplique sur le même nœud n'est pas possible, mais Elasticsearch peut accepter 0 réplique assignée).

⚠️ **Note** : On peut modifier le nombre de répliques d'un index existant sans le supprimer.


In [ ]:
# Mettre à jour le nombre de répliques à 1
es.indices.put_settings(
    index="multimedia",
    body={
        "index": {
            "number_of_replicas": 1
        }
    }
)
print("✅ Facteur de réplication mis à jour à 1")


## 4b. Vérification de l'état du cluster (devrait être Green)

Vérifions à nouveau l'état du cluster. Avec 1 réplique et un seul nœud, le cluster devrait maintenant être en état **green** (ou rester en yellow si la réplique ne peut toujours pas être créée, mais avec moins de shards non assignés).


In [ ]:
# Vérification de la santé du cluster après mise à jour
health = es.cluster.health()
print(f"État du cluster: {health['status'].upper()}")
print(f"Nombre de nœuds: {health['number_of_nodes']}")
print(f"Shards primaires actifs: {health['active_primary_shards']}")
print(f"Shards actifs: {health['active_shards']}")
print(f"Shards non assignés: {health['unassigned_shards']}")

if health['status'] == 'green':
    print("\n🟢 Le cluster est maintenant en état GREEN")
    print("   → Tous les shards primaires et répliques sont actifs")
elif health['status'] == 'yellow':
    print("\n🟡 Le cluster est toujours en état YELLOW")
    print("   → Avec un seul nœud, la réplique ne peut pas être créée")
    print("   → Mais le cluster peut fonctionner normalement")
elif health['status'] == 'red':
    print("\n🔴 Le cluster est en état RED")
    print("   → Certains shards primaires ne sont pas actifs")

pprint(health)


## 5b. Vérification de l'état du cluster (devrait être Red)

Pour observer un état **red**, il faudrait normalement arrêter un nœud qui contient des shards primaires. 

⚠️ **Attention** : Dans un environnement de développement avec un seul nœud, si vous arrêtez ce nœud, vous ne pourrez plus vous connecter à Elasticsearch.

Pour simuler un état red de manière sécurisée, nous pouvons :
1. Vérifier l'état actuel
2. Expliquer comment un état red se produit

**Note** : Si vous avez un cluster multi-nœuds, vous pouvez arrêter un nœud contenant des shards primaires pour observer l'état red.


In [ ]:
# Vérification de la santé du cluster
health = es.cluster.health()
print(f"État du cluster: {health['status'].upper()}")
print(f"Nombre de nœuds: {health['number_of_nodes']}")
print(f"Shards primaires actifs: {health['active_primary_shards']}")
print(f"Shards actifs: {health['active_shards']}")
print(f"Shards non assignés: {health['unassigned_shards']}")

if health['status'] == 'red':
    print("\n🔴 Le cluster est en état RED")
    print("   → Certains shards primaires ne sont pas actifs")
    print("   → Cela signifie qu'une partie des données n'est pas accessible")
elif health['status'] == 'yellow':
    print("\n🟡 Le cluster est en état YELLOW")
    print("   → Tous les shards primaires sont actifs")
elif health['status'] == 'green':
    print("\n🟢 Le cluster est en état GREEN")
    print("   → Tous les shards primaires et répliques sont actifs")

print("\n" + "="*60)
print("Pour observer un état RED dans un environnement de test:")
print("1. Avoir un cluster avec au moins 2 nœuds")
print("2. Arrêter un nœud qui contient des shards primaires")
print("3. Le cluster passera en état RED car les shards primaires")
print("   de ce nœud ne seront plus accessibles")
print("="*60)

pprint(health)


## Informations supplémentaires sur l'index

Vérifions les détails de l'index "multimedia" pour voir sa configuration actuelle.


In [ ]:
stats['_all']['total']

In [ ]:
# Obtenir les paramètres de l'index
settings = es.indices.get_settings(index="multimedia")
print("Paramètres de l'index 'multimedia':")
pprint(settings['multimedia']['settings']['index'])

# Obtenir les statistiques de l'index
stats = es.indices.stats(index="multimedia")
print("\nStatistiques de l'index 'multimedia':")
print(f"Nombre de shards primaires: {stats['indices']['multimedia']['primaries']['docs']['count']}")
print(f"Nombre total de shards: {stats['_shards']['total']}")
print(f"Shards réussis: {stats['_shards']['successful']}")
print(f"Shards échoués: {stats['_shards']['failed']}")

## Nettoyage

Supprimons l'index pour nettoyer (optionnel).


In [ ]:
# Supprimer l'index multimedia
if es.indices.exists(index="multimedia"):
    es.indices.delete(index="multimedia")
    print("🗑️  Index 'multimedia' supprimé")
else:
    print("ℹ️  L'index 'multimedia' n'existe pas")


## Résumé des opérations

Ce notebook a couvert les concepts suivants sur Elasticsearch :

1. ✅ **Création d'index avec facteur de réplication**
   - Comprendre l'impact du nombre de répliques sur l'état du cluster

2. ✅ **États du cluster**
   - 🟢 **Green** : Tous les shards primaires et répliques sont actifs
   - 🟡 **Yellow** : Tous les shards primaires sont actifs, mais certaines répliques ne le sont pas
   - 🔴 **Red** : Certains shards primaires ne sont pas actifs

3. ✅ **Modification dynamique des paramètres**
   - Mise à jour du nombre de répliques sans supprimer l'index

4. ✅ **Monitoring du cluster**
   - Vérification de la santé du cluster
   - Compréhension des métriques (shards actifs, non assignés, etc.)

### Points clés à retenir :

- **Répliques** : Une réplique doit être sur un nœud différent du shard primaire
- **Cluster à un seul nœud** : Ne peut pas avoir de répliques, donc état souvent **yellow**
- **Cluster multi-nœuds** : Permet d'avoir des répliques et d'atteindre l'état **green**
- **État red** : Se produit quand des shards primaires ne sont plus accessibles (nœud arrêté, disque plein, etc.)
